# Gravitation constrained to a surface

Consider a point mass $m$ constrained to a surface in the 3-D space. The system is submitted to a uniform constant acceleration $g$ and moves without friction. For simplicity, we assume that the surface is stable by rotation around an axis $\Delta$ parallel to $g$. The referential is taken to be centred on a point $O$ of $\Delta$. Its $z$-axis is taken to be the semi-axis of $\Delta$ in the opposite direction to $g$. The $x$- and $y$-axes are chosen arbitrarily so as to form an orthogonal basis.

The generalised coordinates of the system are the angle $\phi$ ("azimuth") which is the polar coordinate of the projection of the point mass on the $x,y$-plane, and the angle $\theta$ ("zenith") which is the angle of the $z$-axis with the point mass. We have $0\leq\theta\leq\pi$, and any two $\theta,\phi$ identify a single point on the surface at radius $R(\theta)$. The coordinates of the point mass are then given by


\begin{equation*}
x = R\sin\theta\cos\phi
\hspace{2cm}
y = R\sin\theta\sin\phi
\hspace{2cm}
z = -R\cos\theta
\end{equation*}

The only active force is weight with coordinates $(0,0,-mg)$, hence of the form $-\nabla V$ with $V=mgz=-mgR\cos\theta$. Hence the Lagrangian

\begin{eqnarray*}
\mathcal{L} & = & m\left(\frac{1}{2}R^2\dot{\phi}^2\sin^2\theta+\frac{1}{2}(R^2+R'^2)\dot{\theta}^2+gR\cos\theta\right)
\end{eqnarray*}

Introducing the intermediary variable $q=R^2\dot{\phi}\sin^2\theta$ (angular moment), the equations of the dynamics are given by:

\begin{equation*}
\begin{array}{rrcl}
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{\phi}} = \frac{\partial\mathcal{L}}{\partial\phi}\right]
\hspace{1cm} &
\dot{\phi} & = & \frac{q}{R^2\sin^2\theta}
\\
& \dot{q} & = & 0
\\
\left[\frac{\mathbf{d}}{\mathbf{d}t}\frac{\partial\mathcal{L}}{\partial\dot{\theta}} = \frac{\partial\mathcal{L}}{\partial\theta}\right]
\hspace{1cm} &
\ddot{\theta} & = & \frac{1}{R^2+R'^2}\left(\frac{q^2}{R^3\sin^3\theta}(R'\sin\theta+R\cos\theta)+g(R'\cos\theta-R\sin\theta)-R'(R+R'')\dot{\theta}^2\right)
\end{array}
\end{equation*}

For a sphere, $R$ is constant and $R'=R''=0$ and, introducing $Q\triangleq\frac{q}{R^2}$, the system simplifies to

\begin{equation*}
\begin{array}{rcl}
\dot{\phi} & = & \frac{Q}{\sin^2\theta}\\
\dot{Q} & = & 0\\
\ddot{\theta} & = & \frac{Q^2\cos\theta}{\sin^3\theta}-\frac{g}{R}\sin\theta
\end{array}
\end{equation*}

Documentation: [here](../doc/_build/html/odesimu.html)

In [1]:
%pylab
# for inline animation (but slower and more resource consuming):
#%pylab nbagg

import logging
from functools import partial
from ipyshow.util import Setup
from ipyshow.odesimu.system import System
from ipyshow.odesimu.util import logger_hook

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
#----------------------------------------------------------------------------------------------------
class SphereSlider (System):
#----------------------------------------------------------------------------------------------------

    shadowshape = (5,)

    @Setup(
        'R: radius of the constraining sphere [m]',
        'G: intensity of the gravity [m.sec^-2]'
    )
    def __init__(self,R,G):
        from matplotlib.cm import get_cmap
        self.R, self.G = R, G
        def main(t,state,a=G/R):
            θ,dθ,ϕ,q = state
            cosθ,sinθ = cos(θ),sin(θ)
            dϕ = q/square(sinθ)
            ddθ = (square(dϕ)*cosθ-a)*sinθ
            dq = 0
            return array((dθ,ddθ,dϕ,dq))
        self.main = main
        def jac(t,state,a=G/R):
            θ,dθ,ϕ,q = state
            cosθ,sinθ = cos(θ),sin(θ)
            s = 1/square(sinθ)
            v = 2*q*s*cosθ/sinθ
            return array(((0,1,0,0),(square(q)*s*(2-3*s)-a*cosθ,0,0,v),(-v,0,0,s),(0,0,0,0)))
        self.jacobian = jac
        def fordisplay(state,cmap=get_cmap('rainbow')):
            θ,dθ,ϕ,q = state
            r = R*sin(θ)
            live = (r*cos(ϕ),r*sin(ϕ))+cmap((1-cos(θ))/2)[:-1]
            return live, live
        self.fordisplay = fordisplay

    def display(self,ax,refsize=None,**ka):
        from matplotlib.patches import Circle
        R = 1.05*self.R
        ax.set_xlim(-R,R)
        ax.set_ylim(-R,R)
        ax.scatter((0.,),(0.,),c='k',marker='o',s=refsize)
        ax.add_patch(Circle((0,0),self.R,edgecolor='k',facecolor='none'))
        diag_s = ax.scatter((),(),s=refsize,marker='o',c='r')
        tail_l, = ax.plot((),(),'gray')
        tail_s = ax.scatter((),(),s=refsize,marker='o')
        def disp(t,live,tail):
            x,y = live[:2]
            diag_s.set_offsets(((x,y),))
            diag_s.set_color(live[2:])
            tail_l.set_data(tail[:,0],tail[:,1])
            tail_s.set_offsets(tail[:,:2])
            tail_s.set_color(tail[:,2:])
        return super().display(ax,disp,**ka)

    @staticmethod
    @Setup(
        'θ: zenith [deg]',
        'dθ: angular speed (zenith) [deg.sec^-1]',
        'ϕ: azimuth [deg]',
        'dϕ: angular speed (azimuth) [deg.sec^-1]',
        θ=0.,dθ=0.,ϕ=0.,dϕ=0.
    )
    def makestate(θ,dθ,ϕ,dϕ):
        s = sin(radians(θ))
        return radians((θ,dθ,ϕ,dϕ*square(s)))

    @Setup(
        'refsize: size (area) of the blob for display [pt^2]',
        taild=1.,refsize=50.,hooks=(partial(logger_hook,logger=logging.getLogger()),)
    )
    def launch(self,*a,**ka): return super().launch(*a,**ka)

In [3]:
Setup.display(SphereSlider)

**** SphereSlider.__init__(self, R, G) ****
    R         (          ): radius of the constraining sphere [m]
    G         (          ): intensity of the gravity [m.sec^-2]
**** SphereSlider.launch(self, *a, **ka) ****
    maxtime   (inf       ): total simulation time length [sec]
    srate     (25.0      ): sampling rate [sec^-1]
    taild     (1.0       ): shadow duration [sec]
    hooks     ((functools): tuple of display hooks []
    ini       (          ): initial state []
    refsize   (50.0      ): size (area) of the blob for display [pt^2]
**** SphereSlider.makestate(θ, dθ, φ, dφ) ****
    θ         (0.0       ): zenith [deg]
    dθ        (0.0       ): angular speed (zenith) [deg.sec^-1]
    ϕ         (          ): azimuth [deg]
    dϕ        (          ): angular speed (azimuth) [deg.sec^-1]

In [4]:
syst = SphereSlider(R=2.,G=9.81); ini = dict(θ=135,dθ=0,dϕ=-30)
syst.launch(ini=syst.makestate(**ini))